### Сгрузим данные из таблицы info

In [209]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import numpy as np
import pandas as pd

In [210]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] #права на работу с данной таблицей (readonly)

# The ID and range of a sample spreadsheet.
#Напишем функцию для выделения из ссылки на таблицу ее ID
def link_id(link):
    # Поиск подстроки "/d/", которая следует за "spreadsheets.google.com"
    start_idx = link.find("/d/")
    # Находим индекс конца идентификатора таблицы
    end_idx = link.find("/", start_idx + 3)
    # Извлекаем идентификатор таблицы
    sheet_id = link[start_idx + 3:end_idx]
    return sheet_id


def info(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES 
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
        values = result.get("values", [])

        if not values:
            print("No data found.")
            return

        # Извлечение заголовков столбцов
        headers = values[0]

        # Создание словаря для хранения данных по столбцам
        data_dict = {header: [] for header in headers}

        # Заполнение словаря данными из столбцов
        for row in values[1:]:
            for idx, value in enumerate(row):
                data_dict[headers[idx]].append(value)

        # заменим номер у каждой ссылки листа в конце на 0
        def replace_gid(link):
            # Разделение ссылки на две части по параметру gid
            parts = link.split("gid=")
            # Обновление значения параметра gid на 0
            new_link = parts[0] + "gid=0"
            return new_link
        
                # заменим номер у каждой ссылки листа в конце на 0
        for idx, link in enumerate(data_dict['Ссылка на общую таблицу контракта']):
            data_dict['Ссылка на общую таблицу контракта'][idx] = replace_gid(link)
        
        
        print(data_dict)

    except HttpError as err:
        print(err)
    return(data_dict)





In [211]:
if __name__ == "__main__":
    SAMPLE_SPREADSHEET_ID_INFO = "1MhYj1NjS3uv7gOC5JGp0wToC2YNH-6lg_inNSVo3K4o" #ID таблицы info (часть ссылки)
    SAMPLE_RANGE_NAME_INFO = "Сontract info шаблон!A1:E" #задаем лист и рассматриваемые ячейки
    info_dict = info(SAMPLE_SPREADSHEET_ID_INFO, SAMPLE_RANGE_NAME_INFO)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'Название контракта': ['Контракт №1', 'Контракт №1', 'Контракт №1', 'Контракт №1', 'Контракт №2', 'Контракт №2'], 'Ссылка на общую таблицу контракта': ['https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1IUpVTWvWELWM9nGRMyxPKQ8XNBK1YCo0d7mYRW3PFIo/edit#gid=0', 'https://docs.google.com/spreadsheets/d/1IUpVTWvWELWM9nGRMyxPKQ8XNBK1YCo0d7mYRW3PFIo/edit#gid=0'], 'Менеджер контракта': ['Блогеры ищеек', 'Nexus', 'Rahul', 'Крупные интеграции', 'Блогеры ищеек', 'Nexus'], 'Ссылка на лист контракта по типу менеджера исполнителя': ['https://docs.google.com/spreadsheets/d/1Lvxu_Hkcvy2ihnvvQosxoZ6tymAO8EqiYJq66AydEDg/edit#gi

### сгрузим данные из таблиц всех из инфо

In [212]:
#Создадим словарь, с которого будем подгружать данные в таблицу Total
aggregate_tables = {'Ссылка на аккаунт исполнителя':[], 'Тип менеджера исполнителя':[], 'тг id менеджера исполнителя':[], 'Кол-во исполнителей':[], 'тип контракта':[],'Sub ID 2':[], 'Дата выхода интеграции':[], 'Расходы в Rs':[], 'трек ссылка':[]}


In [213]:

SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] #права на работу с данной таблицей (readonly)

#Напишем функцию для выделения из ссылки на таблицу ее ID
def link_id(link):
    # Поиск подстроки "/d/", которая следует за "spreadsheets.google.com"
    start_idx = link.find("/d/")
    # Находим индекс конца идентификатора таблицы
    end_idx = link.find("/", start_idx + 3)
    # Извлекаем идентификатор таблицы
    sheet_id = link[start_idx + 3:end_idx]
    return sheet_id


def table_lists(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES 
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
        values = result.get("values", [])

        if not values:
            print("No data found.")
            return

        # Извлечение заголовков столбцов
        headers = values[0]

        # Создание словаря для хранения данных по столбцам
        data_dict = {header: [] for header in headers}

        # Заполнение словаря данными из столбцов
        for row in values[1:]:
            for idx, value in enumerate(row):
                data_dict[headers[idx]].append(value)
        
        
        

    except HttpError as err:
        print(err)
    return(data_dict)





In [214]:
link = 'https://docs.google.com/spreadsheets/d/1IUpVTWvWELWM9nGRMyxPKQ8XNBK1YCo0d7mYRW3PFIo/edit#gid=1083647941'
sheet_id = link_id(link)
range_name = 'Блогеры ищеек' +'!A1:E'  # Указываем правильный диапазон данных
table = table_lists(sheet_id, range_name)
table

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs',
  'https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs'],
 'тг id менеджера исполнителя': ['9888888888', '11111111111111'],
 'Дата выхода интеграции': ['3/25/2024 17:25:12', '3/25/2024 17:25:12'],
 'Расходы в Rs': ['4287', '4287'],
 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr21&sub_id3=h3',
  'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr22&sub_id3=h3']}

Напишем функцию, которая достает sub_id2 из ссылки

In [215]:
import re

def extract_sub_id2(url):
    # Используем регулярное выражение для поиска подстроки после "sub_id2=" и до символа "&"
    match = re.search(r'sub_id2=([^&]+)', url)
    
    if match:
        # Если подстрока найдена, возвращаем значение после "sub_id2="
        return match.group(1)
    else:
        # Если подстрока не найдена, возвращаем None или пустую строку
        return ""

In [216]:
for elem in range(len(info_dict['Ссылка на общую таблицу контракта'])):

    if info_dict['Менеджер контракта'][elem] == 'Блогеры ищеек':
        #блогеры ищеек
        link = info_dict['Ссылка на общую таблицу контракта'][elem]
        sheet_id = link_id(link)
        range_name = info_dict['Менеджер контракта'][0] +'!A1:E'  # Указываем правильный диапазон данных
        table = table_lists(sheet_id, range_name)
        print(table)  
        for number_link in range(len(table['Ссылка на аккаунт исполнителя'])):
            aggregate_tables['Ссылка на аккаунт исполнителя'].append(table['Ссылка на аккаунт исполнителя'][number_link])
            aggregate_tables['Тип менеджера исполнителя'].append('Блогеры ищеек')
            aggregate_tables['тип контракта'].append(info_dict['Название контракта'][elem])
            aggregate_tables['тг id менеджера исполнителя'].append(table['тг id менеджера исполнителя'][number_link])
            aggregate_tables['Кол-во исполнителей'].append(1)
            aggregate_tables['Дата выхода интеграции'].append(table['Дата выхода интеграции'][number_link])
            aggregate_tables['Расходы в Rs'].append(table['Расходы в Rs'][number_link])
            aggregate_tables['трек ссылка'].append(table['трек ссылка'][number_link])
            aggregate_tables['Sub ID 2'].append(extract_sub_id2(table['трек ссылка'][number_link]))

    elif info_dict['Менеджер контракта'][elem] == 'Крупные интеграции':
        # крупные интеграции
        link = info_dict['Ссылка на общую таблицу контракта'][elem]
        sheet_id = link_id(link)
        range_name = info_dict['Менеджер контракта'][elem] +'!A1:E'  # Указываем правильный диапазон данных
        table = table_lists(sheet_id, range_name)
        print(table)
        aggregate_tables['Ссылка на аккаунт исполнителя'].append(table['Ссылка на аккаунт исполнителя'][0])
        aggregate_tables['Тип менеджера исполнителя'].append('Крупные интеграции')
        aggregate_tables['тип контракта'].append(info_dict['Название контракта'][elem])
        aggregate_tables['тг id менеджера исполнителя'].append(table['тг id менеджера исполнителя'][0])
        aggregate_tables['Кол-во исполнителей'].append(1)
        aggregate_tables['Дата выхода интеграции'].append(table['Дата выхода интеграции'][0])
        aggregate_tables['Расходы в Rs'].append(table['Расходы в Rs'][0])
        aggregate_tables['трек ссылка'].append(table['трек ссылка'][0])
        aggregate_tables['Sub ID 2'].append(extract_sub_id2(table['трек ссылка'][0]))

    elif not info_dict['Менеджер контракта'][elem] == 'Крупные интеграции' and  not info_dict['Менеджер контракта'][elem] == 'Блогеры ищеек':
        #агентства
        link = info_dict['Ссылка на общую таблицу контракта'][elem]
        sheet_id = link_id(link)
        range_name = info_dict['Менеджер контракта'][elem] +'!A1:D'  # Указываем правильный диапазон данных
        table = table_lists(sheet_id, range_name)
        print(table)

        aggregate_tables['Ссылка на аккаунт исполнителя'].append([''][0])
        aggregate_tables['Тип менеджера исполнителя'].append('Агентства')
        aggregate_tables['тип контракта'].append(info_dict['Название контракта'][elem])
        aggregate_tables['тг id менеджера исполнителя'].append(table['тг id менеджера исполнителя'][0])
        aggregate_tables['Кол-во исполнителей'].append(table['Кол-во исполнителей'][0])
        aggregate_tables['Дата выхода интеграции'].append([''][0])
        aggregate_tables['Расходы в Rs'].append(table['Расходы в Rs'][0])
        aggregate_tables['трек ссылка'].append(table['трек ссылка'][0])
        aggregate_tables['Sub ID 2'].append(extract_sub_id2(table['трек ссылка'][0]))

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/zakhmi.writess_?igsh=dDlzdTY4aHgzMHU3'], 'тг id менеджера исполнителя': ['999999999999'], 'Дата выхода интеграции': ['3/25/2024 13:40:08'], 'Расходы в Rs': ['7864287'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr11&sub_id3=h257']}


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'тг id менеджера исполнителя': ['77777777'], 'Кол-во исполнителей': ['1672'], 'Расходы в Rs': ['1800000'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=nex1&sub_id3=h3']}


...

{'тг id менеджера исполнителя': ['8888888888'], 'Кол-во исполнителей': ['1089'], 'Расходы в Rs': ['786890'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=rah1&sub_id3=h3']}


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/the_challengers07?igsh=dWdsdmxndnZpcDVm'], 'тг id менеджера исполнителя': ['10000000'], 'Дата выхода интеграции': ['22.05.2024'], 'Расходы в Rs': ['9878759'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr&sub_id3=h98']}


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs', 'https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs'], 'тг id менеджера исполнителя': ['9888888888', '11111111111111'], 'Дата выхода интеграции': ['3/25/2024 17:25:12', '3/25/2024 17:25:12'], 'Расходы в Rs': ['4287', '4287'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr21&sub_id3=h3', 'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr22&sub_id3=h3']}
{'тг id менеджера исполнителя': ['77777777'], 'Кол-во исполнителей': ['2567'], 'Расходы в Rs': ['7000000'], 'трек ссылка': ['https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=nex2&sub_id3=h3']}


In [217]:
print(aggregate_tables)

{'Ссылка на аккаунт исполнителя': ['https://www.instagram.com/zakhmi.writess_?igsh=dDlzdTY4aHgzMHU3', '', '', 'https://www.instagram.com/the_challengers07?igsh=dWdsdmxndnZpcDVm', 'https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs', 'https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs', ''], 'Тип менеджера исполнителя': ['Блогеры ищеек', 'Агентства', 'Агентства', 'Крупные интеграции', 'Блогеры ищеек', 'Блогеры ищеек', 'Агентства'], 'тг id менеджера исполнителя': ['999999999999', '77777777', '8888888888', '10000000', '9888888888', '11111111111111', '77777777'], 'Кол-во исполнителей': [1, '1672', '1089', 1, 1, 1, '2567'], 'тип контракта': ['Контракт №1', 'Контракт №1', 'Контракт №1', 'Контракт №1', 'Контракт №2', 'Контракт №2', 'Контракт №2'], 'Sub ID 2': ['sbr11', 'nex1', 'rah1', 'sbr', 'sbr21', 'sbr22', 'nex2'], 'Дата выхода интеграции': ['3/25/2024 13:40:08', '', '', '22.05.2024', '3/25/2024 17:25:12', '3/25/2024 17:25:12', ''], 'Расходы в Rs': ['7864287', '

__Подгрузим еще аналитику по Аланбес__

Немного преобразуем функцию выгрузки таблицы 

In [218]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] #права на работу с данной таблицей (readonly)

def info_alan(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES 
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
        values = result.get("values", [])

        if not values:
            print("No data found.")
            return

        # Извлечение заголовков столбцов
        headers = values[0]

        # Создание словаря для хранения данных по столбцам
        data_dict = {header: [] for header in headers}

        # Заполнение словаря данными из столбцов
        for row in values[1:]:
            for idx, value in enumerate(row):
                data_dict[headers[idx]].append(value)

        # заменим номер у каждой ссылки листа в конце на 0
        def replace_gid(link):
            # Разделение ссылки на две части по параметру gid
            parts = link.split("gid=")
            # Обновление значения параметра gid на 0
            new_link = parts[0] + "gid=0"
            return new_link
        
        
        print(data_dict)

    except HttpError as err:
        print(err)
    return(data_dict)





In [219]:
if __name__ == "__main__":
    SAMPLE_SPREADSHEET_ID_INFO = "1cmXOesuicSG_1BU7Dzzz4q7UEXpYLCPj81OMNCOnr7w" #ID таблицы info (часть ссылки)
    SAMPLE_RANGE_NAME_INFO = "Выгрузка за неделю!A1:I" #задаем лист и рассматриваемые ячейки
    Alan = info_alan(SAMPLE_SPREADSHEET_ID_INFO, SAMPLE_RANGE_NAME_INFO)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


{'Sub ID 2': ['nex2', 'sbr21', 'nex1', 'rah1', 'sbr11', 'sbr', 'sbr22'], 'Total number of clicks': ['1', '2', '3', '4', '5', '6', '7'], 'Unique clicks count': ['1', '2', '3', '4', '5', '6', '7'], 'Registrations': ['1', '2', '3', '4', '5', '6', '7'], 'FTD': ['1', '2', '3', '4', '5', '6', '7'], 'RD': ['1', '2', '3', '4', '5', '6', '7'], 'DEPOSITS AMOUNT': ['1', '2', '3', '4', '5', '6', '7'], 'Payout': ['1', '2', '3', '4', '5', '6', '7'], 'GGR': ['1', '2', '3', '4', '5', '6', '7']}


КОРОЧЕ НАДО ДОРАБОТАТЬ ВОТ ЭТОТ КОД НИЖЕ, ЧТОБЫ СОПОСТАВЛЯЛИСЬ ДАННЫЕ ДВУХ СЛОВАРЕЙ ПО САБАМ

In [220]:
aggregate_tables_pd = pd.DataFrame(aggregate_tables)
Alan_pd = pd.DataFrame(Alan)
df_manual = pd.merge(aggregate_tables_pd, Alan_pd, on='Sub ID 2')

In [221]:
Alan_pd

,Sub ID 2,Total number of clicks,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR
0,nex2,1,1,1,1,1,1,1,1
1,sbr21,2,2,2,2,2,2,2,2
2,nex1,3,3,3,3,3,3,3,3
3,rah1,4,4,4,4,4,4,4,4
4,sbr11,5,5,5,5,5,5,5,5
5,sbr,6,6,6,6,6,6,6,6
6,sbr22,7,7,7,7,7,7,7,7


In [222]:
aggregate_tables_pd

,Ссылка на аккаунт исполнителя,Тип менеджера исполнителя,тг id менеджера исполнителя,Кол-во исполнителей,тип контракта,Sub ID 2,Дата выхода интеграции,Расходы в Rs,трек ссылка
0,https://www.instagram.com/zakhmi.writess_?igsh...,Блогеры ищеек,999999999999,1,Контракт №1,sbr11,3/25/2024 13:40:08,7864287,https://click.tracktik24.com/DKyNxpZ4?landing=...
1,,Агентства,77777777,1672,Контракт №1,nex1,,1800000,https://click.tracktik24.com/DKyNxpZ4?landing=...
2,,Агентства,8888888888,1089,Контракт №1,rah1,,786890,https://click.tracktik24.com/DKyNxpZ4?landing=...
3,https://www.instagram.com/the_challengers07?ig...,Крупные интеграции,10000000,1,Контракт №1,sbr,22.05.2024,9878759,https://click.tracktik24.com/DKyNxpZ4?landing=...
4,https://www.instagram.com/pahadi_vlogger22?igs...,Блогеры ищеек,9888888888,1,Контракт №2,sbr21,3/25/2024 17:25:12,4287,https://click.tracktik24.com/DKyNxpZ4?landing=...
5,https://www.instagram.com/pahadi_vlogger22?igs...,Блогеры ищеек,11111111111111,1,Контракт №2,sbr22,3/25/2024 17:25:12,4287,https://click.tracktik24.com/DKyNxpZ4?landing=...
6,,Агентства,77777777,2567,Контракт №2,nex2,,7000000,https://click.tracktik24.com/DKyNxpZ4?landing=...


In [223]:
df_manual.fillna("None", inplace=True)
df_manual

,Ссылка на аккаунт исполнителя,Тип менеджера исполнителя,тг id менеджера исполнителя,Кол-во исполнителей,тип контракта,Sub ID 2,Дата выхода интеграции,Расходы в Rs,трек ссылка,Total number of clicks,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR
0,https://www.instagram.com/zakhmi.writess_?igsh...,Блогеры ищеек,999999999999,1,Контракт №1,sbr11,3/25/2024 13:40:08,7864287,https://click.tracktik24.com/DKyNxpZ4?landing=...,5,5,5,5,5,5,5,5
1,,Агентства,77777777,1672,Контракт №1,nex1,,1800000,https://click.tracktik24.com/DKyNxpZ4?landing=...,3,3,3,3,3,3,3,3
2,,Агентства,8888888888,1089,Контракт №1,rah1,,786890,https://click.tracktik24.com/DKyNxpZ4?landing=...,4,4,4,4,4,4,4,4
3,https://www.instagram.com/the_challengers07?ig...,Крупные интеграции,10000000,1,Контракт №1,sbr,22.05.2024,9878759,https://click.tracktik24.com/DKyNxpZ4?landing=...,6,6,6,6,6,6,6,6
4,https://www.instagram.com/pahadi_vlogger22?igs...,Блогеры ищеек,9888888888,1,Контракт №2,sbr21,3/25/2024 17:25:12,4287,https://click.tracktik24.com/DKyNxpZ4?landing=...,2,2,2,2,2,2,2,2
5,https://www.instagram.com/pahadi_vlogger22?igs...,Блогеры ищеек,11111111111111,1,Контракт №2,sbr22,3/25/2024 17:25:12,4287,https://click.tracktik24.com/DKyNxpZ4?landing=...,7,7,7,7,7,7,7,7
6,,Агентства,77777777,2567,Контракт №2,nex2,,7000000,https://click.tracktik24.com/DKyNxpZ4?landing=...,1,1,1,1,1,1,1,1


In [224]:
def to_transport(df_manual):
    # Транспонирование DataFrame и преобразование в словарь
    df_manual_transposed = df_manual.transpose()
    aggregate_tables = df_manual_transposed.to_dict()

    # Преобразование словаря в список списков
    aggregate_tables_result = []
    for key in aggregate_tables.keys():
        aggregate_row = []
        for sub_key in aggregate_tables[key]:
            aggregate_row.append(aggregate_tables[key][sub_key])
        aggregate_tables_result.append(aggregate_row)
    aggregate_tables_result
    return(aggregate_tables_result)
    
aggregate_tables_result = to_transport(df_manual)
aggregate_tables_result

[['https://www.instagram.com/zakhmi.writess_?igsh=dDlzdTY4aHgzMHU3',
  'Блогеры ищеек',
  '999999999999',
  1,
  'Контракт №1',
  'sbr11',
  '3/25/2024 13:40:08',
  '7864287',
  'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr11&sub_id3=h257',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5'],
 ['',
  'Агентства',
  '77777777',
  '1672',
  'Контракт №1',
  'nex1',
  '',
  '1800000',
  'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=nex1&sub_id3=h3',
  '3',
  '3',
  '3',
  '3',
  '3',
  '3',
  '3',
  '3'],
 ['',
  'Агентства',
  '8888888888',
  '1089',
  'Контракт №1',
  'rah1',
  '',
  '786890',
  'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=rah1&sub_id3=h3',
  '4',
  '4',
  '4',
  '4',
  '4',
  '4',
  '4',
  '4'],
 ['https://www.instagram.com/the_challengers07?igsh=dWdsdmxndnZpcDVm',
  'Крупные интеграции',
  '10000000',
  1,
  'Контракт №1',
  'sbr',
  '22.05.2024',
  '9878759',
  'https://click.tracktik

### Загрузка данные в таблицу Total

In [225]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os.path
import logging

# Настройка логгера
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] #права на работу с данной таблицей (readonly)

def update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, values):
    """Update values in a sample spreadsheet."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES 
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)

        # Очистка диапазона перед добавлением новой таблицы
        clear_range(service, SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME)

        data = [{
            'range': SAMPLE_RANGE_NAME,
            'values': values
        }]
        body = {
            'valueInputOption': 'USER_ENTERED',
            'data': data
        }
        logger.info(f"Отправка запроса обновления данных: {body}")  # Логируем данные запроса
        result = service.spreadsheets().values().batchUpdate(spreadsheetId=SAMPLE_SPREADSHEET_ID, body=body).execute()
        logger.info(f"Получен результат: {result}")  # Логируем результат запроса
        print('{0} cells updated.'.format(result.get('totalUpdatedCells')))
        
    except Exception as err:
        logger.error(f"Произошла ошибка: {err}")  # Логируем ошибку

def clear_range(service, spreadsheet_id, range_name):
    """Clears the specified range of values in a spreadsheet."""
    request = service.spreadsheets().values().clear(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        body={}
    )
    response = request.execute()

def main():
    SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' # Замените на ID вашей таблицы
    SAMPLE_RANGE_NAME = 'Общая выгрузка!A2:Q' # Замените на нужный вам диапазон
    values = aggregate_tables_result # Замените на свои данные
    update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, values)

if __name__ == '__main__':
    main()  


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Общая выгрузка!A2:Q', 'values': [['https://www.instagram.com/zakhmi.writess_?igsh=dDlzdTY4aHgzMHU3', 'Блогеры ищеек', '999999999999', 1, 'Контракт №1', 'sbr11', '3/25/2024 13:40:08', '7864287', 'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=sbr11&sub_id3=h257', '5', '5', '5', '5', '5', '5', '5', '5'], ['', 'Агентства', '77777777', '1672', 'Контракт №1', 'nex1', '', '1800000', 'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=nex1&sub_id3=h3', '3', '3', '3', '3', '3', '3', '3', '3'], ['', 'Агентства', '8888888888', '1089', 'Контракт №1', 'rah1', '', '786890', 'https://click.tracktik24.com/DKyNxpZ4?landing=57&sub_id1=holi&sub_id2=rah1&sub_id3=h3', '4', '4', '4', '4', '4', '4', '4', '4'], ['https://www.instagram.com/the_challengers07?igsh=dWdsdmxndnZp

119 cells updated.


# Вывод аналитики

In [226]:
df_manual.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Ссылка на аккаунт исполнителя  7 non-null      object
 1   Тип менеджера исполнителя      7 non-null      object
 2   тг id менеджера исполнителя    7 non-null      object
 3   Кол-во исполнителей            7 non-null      object
 4   тип контракта                  7 non-null      object
 5   Sub ID 2                       7 non-null      object
 6   Дата выхода интеграции         7 non-null      object
 7   Расходы в Rs                   7 non-null      object
 8   трек ссылка                    7 non-null      object
 9   Total number of clicks         7 non-null      object
 10  Unique clicks count            7 non-null      object
 11  Registrations                  7 non-null      object
 12  FTD                            7 non-null      object
 13  RD       

### Заполним в листе 'Анализ менеджеров и испольнителей контрактов' левый три основные мини таблицы

Напишем функцию, которая считает метрики

In [278]:
# Переименовываем одну из переменных с именем metrics
metrics_list = ['Unique click price', 'Registration price', 
                'deposit price', 'FTD price', 'Unique clicks per blogger', 
                'Unique clicks/ registration', 'Unique clicks count/FTD', 'Registrations/FTD', 
                'average deposit']

def calculate_metrics(sum_table):
    metrics_dict = {}
    metrics_dict['Unique click price'] = sum_table['Расходы в Rs'] / sum_table['Unique clicks count']
    metrics_dict['Registration price'] = sum_table['Расходы в Rs'] / sum_table['Registrations']
    metrics_dict['deposit price'] = sum_table['Расходы в Rs'] / (sum_table['FTD'] + sum_table['RD'])
    metrics_dict['FTD price'] = sum_table['Расходы в Rs'] / sum_table['FTD']
    metrics_dict['Unique clicks per blogger'] = sum_table['Unique clicks count'] / sum_table['Кол-во исполнителей']
    metrics_dict['Unique clicks/ registration'] = sum_table['Unique clicks count'] / sum_table['Registrations']
    metrics_dict['Unique clicks count/FTD'] = sum_table['Unique clicks count'] / sum_table['FTD']
    metrics_dict['Registrations/FTD'] = sum_table['Registrations'] / sum_table['FTD']
    metrics_dict['average deposit'] = sum_table['DEPOSITS AMOUNT'] / (sum_table['FTD'] + sum_table['RD'])
    return metrics_dict

In [279]:
for elem in ['Кол-во исполнителей', 'Total number of clicks', 'Unique clicks count', 'Registrations', 'FTD', 'RD', 'DEPOSITS AMOUNT', 'Payout', 'GGR', 'Расходы в Rs']:
    df_manual[elem] = pd.to_numeric(df_manual[elem])

In [280]:
#создание словаря для итога Total
anaitics_total = df_manual[['Кол-во исполнителей', 'Total number of clicks', 'Unique clicks count', 'Registrations', 'FTD', 'RD', 'DEPOSITS AMOUNT', 'Payout', 'GGR', 'Расходы в Rs']]
anaitics_total = anaitics_total.sum()

anaitics_total_combined = pd.DataFrame([anaitics_total.values], columns=anaitics_total.index)

# преобразуем в нужный формат
anaitics_total = pd.DataFrame(anaitics_total).transpose()
anaitics_total = to_transport(anaitics_total)



# Определение ID таблицы и диапазона
SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' 
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!B3:L3'

# Вызов функции update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, anaitics_total)










INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!B3:L3', 'values': [[5332, 28, 28, 28, 28, 28, 28, 28, 28, 27338510]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 1, 'totalUpdatedColumns': 10, 'totalUpdatedCells': 10, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей контрактов'!B3:K3", 'updatedRows': 1, 'updatedColumns': 10, 'updatedCells': 10}]}


10 cells updated.


In [281]:
metrics_total = calculate_metrics(anaitics_total_combined.reset_index())

metrics_total = metrics_total.values()
# преобразуем в нужный формат
metrics_total = pd.DataFrame(metrics_total).transpose()
metrics_total = to_transport(metrics_total)

SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' # Замените на ID вашей таблицы
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!B6:J6' # Замените на нужный вам диапазон

# Вызов функции update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, metrics_total)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!B6:J6', 'values': [[976375.3571428572, 976375.3571428572, 488187.6785714286, 976375.3571428572, 0.005251312828207052, 1.0, 1.0, 1.0, 0.5]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 1, 'totalUpdatedColumns': 9, 'totalUpdatedCells': 9, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей контрактов'!B6:J6", 'updatedRows': 1, 'updatedColumns': 9, 'updatedCells': 9}]}


9 cells updated.


Добавление аналитики по агентствам

In [282]:
#создание словаря для итога Агентств
anaitics_agences = analitics_total_data[analitics_total_data['Тип менеджера исполнителя'] == 'Агентства'][['Кол-во исполнителей', 'Total number of clicks', 'Unique clicks count', 'Registrations', 'FTD', 'RD', 'DEPOSITS AMOUNT', 'Payout', 'GGR', 'Расходы в Rs']]
anaitics_agences = anaitics_agences.sum()

anaitics_agences_combined = pd.DataFrame([anaitics_agences.values], columns=anaitics_agences.index)

# преобразуем в нужный формат
anaitics_agences = pd.DataFrame(anaitics_agences).transpose()
anaitics_agences = to_transport(anaitics_agences)



SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' # Замените на ID вашей таблицы
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!B11:L11' # Замените на нужный вам диапазон

# Вызов функции update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, anaitics_agences)


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!B11:L11', 'values': [[5328, 8, 8, 8, 8, 8, 8, 8, 8, 9586890]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 1, 'totalUpdatedColumns': 10, 'totalUpdatedCells': 10, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей контрактов'!B11:K11", 'updatedRows': 1, 'updatedColumns': 10, 'updatedCells': 10}]}


10 cells updated.


In [283]:
metrics_agences = calculate_metrics(anaitics_agences_combined.reset_index())

metrics_agences = metrics_agences.values()
# преобразуем в нужный формат
metrics_agences = pd.DataFrame(metrics_agences).transpose()
metrics_agences = to_transport(metrics_agences)

SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' # Замените на ID вашей таблицы
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!B14:J14' # Замените на нужный вам диапазон

# Вызов функции update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, metrics_agences)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!B14:J14', 'values': [[1198361.25, 1198361.25, 599180.625, 1198361.25, 0.0015015015015015015, 1.0, 1.0, 1.0, 0.5]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 1, 'totalUpdatedColumns': 9, 'totalUpdatedCells': 9, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей контрактов'!B14:J14", 'updatedRows': 1, 'updatedColumns': 9, 'updatedCells': 9}]}


9 cells updated.


Добавление аналитик по ищейкам

In [333]:
#создание словаря для итога Агентств
anaitics_tracker = analitics_total_data[analitics_total_data['Тип менеджера исполнителя'] == 'Блогеры ищеек'][['Кол-во исполнителей', 'Total number of clicks', 'Unique clicks count', 'Registrations', 'FTD', 'RD', 'DEPOSITS AMOUNT', 'Payout', 'GGR', 'Расходы в Rs']]
anaitics_tracker = anaitics_tracker.sum()

anaitics_tracker_combined = pd.DataFrame([anaitics_tracker.values], columns=anaitics_tracker.index)

# преобразуем в нужный формат
anaitics_tracker = pd.DataFrame(anaitics_tracker).transpose()
anaitics_tracker = to_transport(anaitics_tracker)



SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' # Замените на ID вашей таблицы
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!B19:L19' # Замените на нужный вам диапазон

# Вызов функции update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, anaitics_tracker)


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!B19:L19', 'values': [[3, 14, 14, 14, 14, 14, 14, 14, 14, 7872861]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 1, 'totalUpdatedColumns': 10, 'totalUpdatedCells': 10, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей контрактов'!B19:K19", 'updatedRows': 1, 'updatedColumns': 10, 'updatedCells': 10}]}


10 cells updated.


In [337]:
metrics_tracker = calculate_metrics(anaitics_tracker_combined.reset_index())

etrics_tracker = metrics_tracker.values()
metrics_tracker

{'Unique click price': 0    562347.214286
 dtype: float64,
 'Registration price': 0    562347.214286
 dtype: float64,
 'deposit price': 0    281173.607143
 dtype: float64,
 'FTD price': 0    562347.214286
 dtype: float64,
 'Unique clicks per blogger': 0    4.666667
 dtype: float64,
 'Unique clicks/ registration': 0    1.0
 dtype: float64,
 'Unique clicks count/FTD': 0    1.0
 dtype: float64,
 'Registrations/FTD': 0    1.0
 dtype: float64,
 'average deposit': 0    0.5
 dtype: float64}

In [289]:
metrics_tracker = calculate_metrics(anaitics_tracker_combined.reset_index())

metrics_tracker = metrics_tracker.values()
# преобразуем в нужный формат
metrics_tracker = pd.DataFrame(metrics_tracker).transpose()
metrics_tracker = to_transport(metrics_tracker)

SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM' # Замените на ID вашей таблицы
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!B22:J22' # Замените на нужный вам диапазон

# Вызов функции update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, metrics_tracker)



INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!B22:J22', 'values': [[562347.2142857143, 562347.2142857143, 281173.60714285716, 562347.2142857143, 4.666666666666667, 1.0, 1.0, 1.0, 0.5]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 1, 'totalUpdatedColumns': 9, 'totalUpdatedCells': 9, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей контрактов'!B22:J22", 'updatedRows': 1, 'updatedColumns': 9, 'updatedCells': 9}]}


9 cells updated.


### Заполним в листе 'Анализ менеджеров и испольнителей контрактов' cтатистика по конверсии агентств

Удалим все ненужные столбцы

In [417]:
stat_agences = df_manual.copy()
stat_agences.drop('Ссылка на аккаунт исполнителя', axis = 1, inplace =True)
stat_agences.drop('Дата выхода интеграции', axis = 1, inplace =True)
stat_agences.drop('тип контракта', axis = 1, inplace =True)
stat_agences.drop('Sub ID 2', axis = 1, inplace =True)
stat_agences.drop('трек ссылка', axis = 1, inplace =True)
stat_agences = stat_agences[stat_agences['Тип менеджера исполнителя'] == 'Агентства']
stat_agences = stat_agences.groupby('Тип менеджера исполнителя').sum().reset_index()
stat_agences.drop('Тип менеджера исполнителя', axis = 1, inplace =True)
stat_agences.drop('Total number of clicks', axis = 1, inplace =True)
stat_agences


,тг id менеджера исполнителя,Кол-во исполнителей,Расходы в Rs,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR
0,77777777888888888877777777,5328,9586890,8,8,8,8,8,8,8


Создадим метрики

In [370]:
stat_agences['Unique click price'] = stat_agences['Расходы в Rs'] / stat_agences['Unique clicks count']
stat_agences['Registration price'] = stat_agences['Расходы в Rs'] / stat_agences['Registrations']
stat_agences['deposit price'] = stat_agences['Расходы в Rs'] / (stat_agences['FTD'] + stat_agences['RD'])
stat_agences['FTD price'] = stat_agences['Расходы в Rs'] / stat_agences['FTD']
stat_agences['Unique clicks per blogger'] = stat_agences['Unique clicks count'] / stat_agences['Кол-во исполнителей']
stat_agences['Unique clicks/ registration'] = stat_agences['Unique clicks count'] / stat_agences['Registrations']
stat_agences['Unique clicks count/FTD'] = stat_agences['Unique clicks count'] / stat_agences['FTD']
stat_agences['Registrations/FTD'] = stat_agences['Registrations'] / stat_agences['FTD']
stat_agences['average deposit'] = stat_agences['DEPOSITS AMOUNT'] / (stat_agences['FTD'] + stat_agences['RD'])
stat_agences


,тг id менеджера исполнителя,Кол-во исполнителей,Расходы в Rs,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR,Unique click price,Registration price,deposit price,FTD price,Unique clicks per blogger,Unique clicks/ registration,Unique clicks count/FTD,Registrations/FTD,average deposit
1,77777777,1672,1800000,3,3,3,3,3,3,3,600000.0,600000.0,300000.00,600000.0,0.001794,1.0,1.0,1.0,0.5
2,8888888888,1089,786890,4,4,4,4,4,4,4,196722.5,196722.5,98361.25,196722.5,0.003673,1.0,1.0,1.0,0.5
6,77777777,2567,7000000,1,1,1,1,1,1,1,7000000.0,7000000.0,3500000.00,7000000.0,0.000390,1.0,1.0,1.0,0.5


Добавим в таблицу (что то функция нормально не работала по формирования данных к экспорту в файл. Перепишу функцию)

__Сюда бы еще добавить полную очистку колонок этой части__

In [376]:
def to_transport(df_manual):
    # Транспонирование DataFrame и преобразование в словарь
    df_manual_transposed = df_manual.transpose()
    aggregate_tables = df_manual_transposed.to_dict()

    # Преобразование словаря в список списков
    aggregate_tables_result = []
    for key in aggregate_tables.keys():
        aggregate_row = []
        for sub_key in aggregate_tables[key]:
            aggregate_row.append(aggregate_tables[key][sub_key])
        aggregate_tables_result.append(aggregate_row)
    return aggregate_tables_result


In [377]:
stat_agences = pd.DataFrame(stat_agences).transpose()

stat_agences = to_transport(stat_agences)

SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM'
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!O3'

# Вызываем функцию update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, stat_agences)


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!O3', 'values': [['77777777', 1672, 1800000, 3, 3, 3, 3, 3, 3, 3, 600000.0, 600000.0, 300000.0, 600000.0, 0.0017942583732057417, 1.0, 1.0, 1.0, 0.5], ['8888888888', 1089, 786890, 4, 4, 4, 4, 4, 4, 4, 196722.5, 196722.5, 98361.25, 196722.5, 0.0036730945821854912, 1.0, 1.0, 1.0, 0.5], ['77777777', 2567, 7000000, 1, 1, 1, 1, 1, 1, 1, 7000000.0, 7000000.0, 3500000.0, 7000000.0, 0.00038955979742890534, 1.0, 1.0, 1.0, 0.5]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 3, 'totalUpdatedColumns': 19, 'totalUpdatedCells': 57, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ менеджеров и испольнителей кон

57 cells updated.


### Статистика по конверсии блогеров-ищеек

Удалим все ненужные столбцы

In [406]:
stat_tracker = df_manual.copy()
stat_tracker.drop('Дата выхода интеграции', axis = 1, inplace =True)
stat_tracker.drop('Кол-во исполнителей', axis = 1, inplace =True)
stat_tracker.drop('тип контракта', axis = 1, inplace =True)
stat_tracker.drop('Sub ID 2', axis = 1, inplace =True)
stat_tracker.drop('трек ссылка', axis = 1, inplace =True)
stat_tracker = stat_tracker[stat_tracker['Тип менеджера исполнителя'] == 'Блогеры ищеек']
stat_tracker = stat_tracker.groupby('Тип менеджера исполнителя').sum().reset_index()
stat_tracker.drop('Тип менеджера исполнителя', axis = 1, inplace =True)
stat_tracker.drop('Total number of clicks', axis = 1, inplace =True)
stat_tracker

,Ссылка на аккаунт исполнителя,тг id менеджера исполнителя,Расходы в Rs,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR
0,https://www.instagram.com/zakhmi.writess_?igsh...,999999999999988888888811111111111111,7872861,14,14,14,14,14,14,14


In [391]:

stat_tracker['Unique click price'] = stat_tracker['Расходы в Rs'] / stat_tracker['Unique clicks count']
stat_tracker['Registration price'] = stat_tracker['Расходы в Rs'] / stat_tracker['Registrations']
stat_tracker['deposit price'] = stat_tracker['Расходы в Rs'] / (stat_tracker['FTD'] + stat_tracker['RD'])
stat_tracker['FTD price'] = stat_tracker['Расходы в Rs'] / stat_tracker['FTD']
stat_tracker['Unique clicks/ registration'] = stat_tracker['Unique clicks count'] / stat_tracker['Registrations']
stat_tracker['Unique clicks count/FTD'] = stat_tracker['Unique clicks count'] / stat_tracker['FTD']
stat_tracker['Registrations/FTD'] = stat_tracker['Registrations'] / stat_tracker['FTD']
stat_tracker['average deposit'] = stat_tracker['DEPOSITS AMOUNT'] / (stat_tracker['FTD'] + stat_tracker['RD'])
stat_tracker

,Ссылка на аккаунт исполнителя,тг id менеджера исполнителя,Расходы в Rs,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR,Unique click price,Registration price,deposit price,FTD price,Unique clicks/ registration,Unique clicks count/FTD,Registrations/FTD,average deposit
0,https://www.instagram.com/zakhmi.writess_?igsh...,999999999999,7864287,5,5,5,5,5,5,5,1.572857e+06,1.572857e+06,786428.700000,1.572857e+06,1.0,1.0,1.0,0.5
4,https://www.instagram.com/pahadi_vlogger22?igs...,9888888888,4287,2,2,2,2,2,2,2,2.143500e+03,2.143500e+03,1071.750000,2.143500e+03,1.0,1.0,1.0,0.5
5,https://www.instagram.com/pahadi_vlogger22?igs...,11111111111111,4287,7,7,7,7,7,7,7,6.124286e+02,6.124286e+02,306.214286,6.124286e+02,1.0,1.0,1.0,0.5


Тут тоже почему то если не прописать заново функцию, то не работает нормально, но так как это не стоит труда, то пропишем

In [393]:
def to_transport(df_manual):
    # Транспонирование DataFrame и преобразование в словарь
    df_manual_transposed = df_manual.transpose()
    aggregate_tables = df_manual_transposed.to_dict()

    # Преобразование словаря в список списков
    aggregate_tables_result = []
    for key in aggregate_tables.keys():
        aggregate_row = []
        for sub_key in aggregate_tables[key]:
            aggregate_row.append(aggregate_tables[key][sub_key])
        aggregate_tables_result.append(aggregate_row)
    return aggregate_tables_result

In [394]:
stat_tracker = pd.DataFrame(stat_tracker).transpose()
stat_tracker = to_transport(stat_tracker)

SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM'
SAMPLE_RANGE_NAME = 'Анализ менеджеров и испольнителей контрактов!AJ3'

# Вызываем функцию update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, stat_tracker)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ менеджеров и испольнителей контрактов!AJ3', 'values': [['https://www.instagram.com/zakhmi.writess_?igsh=dDlzdTY4aHgzMHU3', '999999999999', 7864287, 5, 5, 5, 5, 5, 5, 5, 1572857.4, 1572857.4, 786428.7, 1572857.4, 1.0, 1.0, 1.0, 0.5], ['https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs', '9888888888', 4287, 2, 2, 2, 2, 2, 2, 2, 2143.5, 2143.5, 1071.75, 2143.5, 1.0, 1.0, 1.0, 0.5], ['https://www.instagram.com/pahadi_vlogger22?igsh=cGx2cjNreXVuZ3Vs', '11111111111111', 4287, 7, 7, 7, 7, 7, 7, 7, 612.4285714285714, 612.4285714285714, 306.2142857142857, 612.4285714285714, 1.0, 1.0, 1.0, 0.5]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 3, 'totalUpdatedColumns': 18, 'totalUpdatedCells': 54, 'totalUpdat

54 cells updated.


### Статистика по конверсии контрактов

Удалим все ненужные столбцы

In [427]:
stat_tracker = df_manual.copy()
stat_tracker.drop('Дата выхода интеграции', axis = 1, inplace =True)
stat_tracker.drop('Sub ID 2', axis = 1, inplace =True)
stat_tracker.drop('трек ссылка', axis = 1, inplace =True)
stat_tracker.drop('Total number of clicks', axis = 1, inplace =True)
stat_tracker.drop('Ссылка на аккаунт исполнителя', axis = 1, inplace =True)
stat_tracker.drop('Тип менеджера исполнителя', axis = 1, inplace =True)
stat_tracker.drop('тг id менеджера исполнителя', axis = 1, inplace =True)
stat_tracker = stat_tracker.groupby('тип контракта').sum().reset_index()
stat_tracker

,тип контракта,Кол-во исполнителей,Расходы в Rs,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR
0,Контракт №1,2763,20329936,18,18,18,18,18,18,18
1,Контракт №2,2569,7008574,10,10,10,10,10,10,10


Введем метрики

In [429]:
stat_tracker['Unique click price'] = stat_tracker['Расходы в Rs'] / stat_tracker['Unique clicks count']
stat_tracker['Registration price'] = stat_tracker['Расходы в Rs'] / stat_tracker['Registrations']
stat_tracker['deposit price'] = stat_tracker['Расходы в Rs'] / (stat_tracker['FTD'] + stat_tracker['RD'])
stat_tracker['FTD price'] = stat_tracker['Расходы в Rs'] / stat_tracker['FTD']
stat_tracker['Unique clicks per blogger'] = stat_tracker['Unique clicks count'] / stat_tracker['Кол-во исполнителей']
stat_tracker['Unique clicks/ registration'] = stat_tracker['Unique clicks count'] / stat_tracker['Registrations']
stat_tracker['Unique clicks count/FTD'] = stat_tracker['Unique clicks count'] / stat_tracker['FTD']
stat_tracker['Registrations/FTD'] = stat_tracker['Registrations'] / stat_tracker['FTD']
stat_tracker['average deposit'] = stat_tracker['DEPOSITS AMOUNT'] / (stat_tracker['FTD'] + stat_tracker['RD'])
stat_tracker

,тип контракта,Кол-во исполнителей,Расходы в Rs,Unique clicks count,Registrations,FTD,RD,DEPOSITS AMOUNT,Payout,GGR,Unique click price,Registration price,deposit price,FTD price,Unique clicks/ registration,Unique clicks count/FTD,Registrations/FTD,average deposit,Unique clicks per blogger
0,Контракт №1,2763,20329936,18,18,18,18,18,18,18,1.129441e+06,1.129441e+06,564720.444444,1.129441e+06,1.0,1.0,1.0,0.5,0.006515
1,Контракт №2,2569,7008574,10,10,10,10,10,10,10,7.008574e+05,7.008574e+05,350428.700000,7.008574e+05,1.0,1.0,1.0,0.5,0.003893


In [430]:
stat_tracker = pd.DataFrame(stat_tracker).transpose()
stat_tracker = to_transport(stat_tracker)

SAMPLE_SPREADSHEET_ID = '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM'
SAMPLE_RANGE_NAME = 'Анализ контрактов!A3'

# Вызываем функцию update_table() для отправки данных в таблицу
update_table(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, stat_tracker)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:__main__:Отправка запроса обновления данных: {'valueInputOption': 'USER_ENTERED', 'data': [{'range': 'Анализ контрактов!A3', 'values': [['Контракт №1', 'Контракт №2'], [2763, 2569], [20329936, 7008574], [18, 10], [18, 10], [18, 10], [18, 10], [18, 10], [18, 10], [18, 10], [1129440.888888889, 700857.4], [1129440.888888889, 700857.4], [564720.4444444445, 350428.7], [1129440.888888889, 700857.4], [1.0, 1.0], [1.0, 1.0], [1.0, 1.0], [0.5, 0.5], [0.006514657980456026, 0.003892565200467108]]}]}
INFO:__main__:Получен результат: {'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'totalUpdatedRows': 19, 'totalUpdatedColumns': 2, 'totalUpdatedCells': 38, 'totalUpdatedSheets': 1, 'responses': [{'spreadsheetId': '1X-THzy2TpMsBAxpR6WlIH1V9TrKgr5kH6ajLJ6fELYM', 'updatedRange': "'Анализ контрактов'!A3:B21", 'updatedRows': 19, 'updatedColumns': 2, 'updatedCells': 38}]}


38 cells updated.


# Моменты по работе по работе с кодом и таблицами по этой части

- каждую неделю перед выгрузкой данных за неделю необходимо 